In [1]:
import pandas as pd 
import duckdb

In [19]:
conn = duckdb.connect('../data/gymBuddy_db.db')

In [20]:
conn.sql("SHOW TABLES")

┌──────────────────┐
│       name       │
│     varchar      │
├──────────────────┤
│ workout          │
│ workout_analysis │
└──────────────────┘

In [21]:
conn.sql("Select max(id) from workout")

┌─────────┐
│ max(id) │
│  int32  │
├─────────┤
│     123 │
└─────────┘

In [22]:
conn.sql("select * from workout where id =123")

┌───────┬──────────────┬────────────────────────────┬──────────┬─────────────────┬───────────────────────┬───────────┐
│  ID   │ workout_name │      timestamp_start       │ rep_goal │ strictness_crit │ strictness_definition │ left_side │
│ int32 │   varchar    │         timestamp          │  int32   │     varchar     │        double         │  boolean  │
├───────┼──────────────┼────────────────────────────┼──────────┼─────────────────┼───────────────────────┼───────────┤
│   123 │ push-ups     │ 2025-06-12 15:11:09.055609 │        1 │ loose           │                  15.0 │ true      │
└───────┴──────────────┴────────────────────────────┴──────────┴─────────────────┴───────────────────────┴───────────┘

In [23]:
conn.sql("SELECT * FROM workout_analysis where workout_id = 123")

┌────────────┬───────┬───────────┬───────────────┬───────────┬─────────┬─────────────┬───────────────────────────────────────────────────────────────────┬───────────────────────┐
│ workout_id │ frame │ timestamp │ series_number │ rep_count │  down   │ form_issues │                            angles_data                            │  ldmrks_of_interest   │
│   int32    │ int32 │ timestamp │     int32     │   int32   │ boolean │   varchar   │ struct("name" varchar, "value" double, joint_indices integer[])[] │ map(varchar, integer) │
├────────────┴───────┴───────────┴───────────────┴───────────┴─────────┴─────────────┴───────────────────────────────────────────────────────────────────┴───────────────────────┤
│                                                                                     0 rows                                                                                     │
└────────────────────────────────────────────────────────────────────────────────────────────────────────

In [19]:
import datetime

In [20]:
array_insert = [{'id': 121, 'frame': 162, 'time': datetime.datetime(2025, 6, 12, 14, 6, 40, 900220), 'rep_count': 0, 'down': True, 'form_issues': 'knees on floor', 'angles_data': '[{"name": "elbow", "value": 129.53147363343317, "joint_indices": [15, 13, 11]}, {"name": "body", "value": 177.76583332780172, "joint_indices": [11, 23, 27]}]', 'ldmrks_keys': ['shoulder', 'elbow', 'wrist', 'hip', 'ankle', 'knee', 'toes'], 'ldmrks_values': [11, 13, 15, 23, 27, 25, 31]}, {'id': 121, 'frame': 163, 'time': datetime.datetime(2025, 6, 12, 14, 6, 41, 22448), 'rep_count': 0, 'down': True, 'form_issues': 'Good form! (angle: 165.9degrees)', 'angles_data': '[{"name": "elbow", "value": 137.23768234411716, "joint_indices": [15, 13, 11]}, {"name": "body", "value": 165.9001494314959, "joint_indices": [11, 23, 27]}]', 'ldmrks_keys': ['shoulder', 'elbow', 'wrist', 'hip', 'ankle', 'knee', 'toes'], 'ldmrks_values': [11, 13, 15, 23, 27, 25, 31]}, {'id': 121, 'frame': 164, 'time': datetime.datetime(2025, 6, 12, 14, 6, 41, 136119), 'rep_count': 1, 'down': False, 'form_issues': 'Good form! (angle: 162.4degrees)', 'angles_data': '[{"name": "elbow", "value": 154.16380858568002, "joint_indices": [15, 13, 11]}, {"name": "body", "value": 162.3936643031923, "joint_indices": [11, 23, 27]}]', 'ldmrks_keys': ['shoulder', 'elbow', 'wrist', 'hip', 'ankle', 'knee', 'toes'], 'ldmrks_values': [11, 13, 15, 23, 27, 25, 31]}]

In [21]:
insert_db = pd.DataFrame(array_insert)

In [75]:
data_to_insert = insert_db.copy()

In [76]:
conn.sql("SHOW workout_analysis")

┌────────────────────┬───────────────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│    column_name     │                            column_type                            │  null   │   key   │ default │  extra  │
│      varchar       │                              varchar                              │ varchar │ varchar │ varchar │ varchar │
├────────────────────┼───────────────────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ workout_id         │ INTEGER                                                           │ NO      │ PRI     │ NULL    │ NULL    │
│ frame              │ INTEGER                                                           │ NO      │ NULL    │ NULL    │ NULL    │
│ timestamp          │ TIMESTAMP                                                         │ NO      │ PRI     │ NULL    │ NULL    │
│ series_number      │ INTEGER                                                     

In [77]:
data_to_insert.rename(columns={'id': 'workout_id','time':'timestamp'}, inplace=True)

In [82]:
{'key': data_to_insert['ldmrks_keys'].values, 'value': data_to_insert['ldmrks_values'].values}

{'key': array([list(['shoulder', 'elbow', 'wrist', 'hip', 'ankle', 'knee', 'toes']),
        list(['shoulder', 'elbow', 'wrist', 'hip', 'ankle', 'knee', 'toes']),
        list(['shoulder', 'elbow', 'wrist', 'hip', 'ankle', 'knee', 'toes'])],
       dtype=object),
 'value': array([list([11, 13, 15, 23, 27, 25, 31]),
        list([11, 13, 15, 23, 27, 25, 31]),
        list([11, 13, 15, 23, 27, 25, 31])], dtype=object)}

In [86]:
data_to_insert['ldmrks_of_interest']= data_to_insert.apply(lambda row:{'key': row['ldmrks_keys'], 'value': row['ldmrks_values']}, axis=1)


In [87]:
data_to_insert['ldmrks_of_interest']

0    {'key': ['shoulder', 'elbow', 'wrist', 'hip', ...
1    {'key': ['shoulder', 'elbow', 'wrist', 'hip', ...
2    {'key': ['shoulder', 'elbow', 'wrist', 'hip', ...
Name: ldmrks_of_interest, dtype: object

In [88]:
data_to_insert

,workout_id,frame,timestamp,rep_count,down,form_issues,angles_data,ldmrks_keys,ldmrks_values,ldmrks_of_interest
0,121,162,2025-06-12 14:06:40.900220,0,True,knees on floor,"[{""name"": ""elbow"", ""value"": 129.53147363343317...","[shoulder, elbow, wrist, hip, ankle, knee, toes]","[11, 13, 15, 23, 27, 25, 31]","{'key': ['shoulder', 'elbow', 'wrist', 'hip', ..."
1,121,163,2025-06-12 14:06:41.022448,0,True,Good form! (angle: 165.9degrees),"[{""name"": ""elbow"", ""value"": 137.23768234411716...","[shoulder, elbow, wrist, hip, ankle, knee, toes]","[11, 13, 15, 23, 27, 25, 31]","{'key': ['shoulder', 'elbow', 'wrist', 'hip', ..."
2,121,164,2025-06-12 14:06:41.136119,1,False,Good form! (angle: 162.4degrees),"[{""name"": ""elbow"", ""value"": 154.16380858568002...","[shoulder, elbow, wrist, hip, ankle, knee, toes]","[11, 13, 15, 23, 27, 25, 31]","{'key': ['shoulder', 'elbow', 'wrist', 'hip', ..."


In [89]:
data_to_insert.drop(columns=['ldmrks_keys', 'ldmrks_values'], inplace=True)

In [90]:
conn.sql("INSERT INTO workout_analysis BY NAME SELECT * FROM data_to_insert")

In [109]:
conn.sql("SELECT * FROM workout_analysis where workout_id = 122")

┌────────────┬───────┬───────────┬───────────────┬───────────┬─────────┬─────────────┬───────────────────────────────────────────────────────────────────┬───────────────────────┐
│ workout_id │ frame │ timestamp │ series_number │ rep_count │  down   │ form_issues │                            angles_data                            │  ldmrks_of_interest   │
│   int32    │ int32 │ timestamp │     int32     │   int32   │ boolean │   varchar   │ struct("name" varchar, "value" double, joint_indices integer[])[] │ map(varchar, integer) │
├────────────┴───────┴───────────┴───────────────┴───────────┴─────────┴─────────────┴───────────────────────────────────────────────────────────────────┴───────────────────────┤
│                                                                                     0 rows                                                                                     │
└────────────────────────────────────────────────────────────────────────────────────────────────────────

In [24]:
conn.close()

In [116]:
raw_landamrks = pd.read_csv('../data/test_landmarks.csv')

In [117]:
import numpy as np
from ast import literal_eval

In [87]:
landmarks = raw_landamrks['landmarks'].apply(lambda x: literal_eval(x))


In [88]:
row = landmarks[0]
len(row)

33

In [96]:
landmarks.apply(lambda x: extend_row(x))[0]

[0.3988448679447174,
 0.2650674879550934,
 0.046302277594804764,
 0.38457638025283813,
 0.2516632378101349,
 0.025139370933175087,
 0.38405337929725647,
 0.2474006712436676,
 0.025041230022907257,
 0.3836759328842163,
 0.2444152534008026,
 0.024942409247159958,
 0.38608673214912415,
 0.252421110868454,
 0.060685254633426666,
 0.38669487833976746,
 0.25013789534568787,
 0.06070072203874588,
 0.3871845304965973,
 0.24670831859111786,
 0.06070617958903313,
 0.3915454149246216,
 0.21023374795913696,
 -0.0355309396982193,
 0.39319393038749695,
 0.21879693865776062,
 0.1233525350689888,
 0.4148765206336975,
 0.2539786994457245,
 0.02702813595533371,
 0.41604915261268616,
 0.25612005591392517,
 0.07387302815914154,
 0.4538514018058777,
 0.20189158618450165,
 -0.1576453447341919,
 0.48180943727493286,
 0.17555245757102966,
 0.2336626499891281,
 0.45799073576927185,
 0.45605185627937317,
 -0.25119683146476746,
 0.4789115786552429,
 0.41498589515686035,
 0.2698378264904022,
 0.4154588580131531,


In [118]:

def extend_row(row):
    lm = []
    for i in range(0, len(row)):
        lm.extend([row[i]['x'], row[i]['y'], row[i]['z']])
    return lm

In [119]:
cols = []
for i in range(0, len(landmarks[0])):
   cols.extend([f'landmark_{i}_x', f'landmark_{i}_y', f'landmark_{i}_z'])

In [120]:
cols

['landmark_0_x',
 'landmark_0_y',
 'landmark_0_z',
 'landmark_1_x',
 'landmark_1_y',
 'landmark_1_z',
 'landmark_2_x',
 'landmark_2_y',
 'landmark_2_z',
 'landmark_3_x',
 'landmark_3_y',
 'landmark_3_z',
 'landmark_4_x',
 'landmark_4_y',
 'landmark_4_z',
 'landmark_5_x',
 'landmark_5_y',
 'landmark_5_z',
 'landmark_6_x',
 'landmark_6_y',
 'landmark_6_z',
 'landmark_7_x',
 'landmark_7_y',
 'landmark_7_z',
 'landmark_8_x',
 'landmark_8_y',
 'landmark_8_z',
 'landmark_9_x',
 'landmark_9_y',
 'landmark_9_z',
 'landmark_10_x',
 'landmark_10_y',
 'landmark_10_z',
 'landmark_11_x',
 'landmark_11_y',
 'landmark_11_z',
 'landmark_12_x',
 'landmark_12_y',
 'landmark_12_z',
 'landmark_13_x',
 'landmark_13_y',
 'landmark_13_z',
 'landmark_14_x',
 'landmark_14_y',
 'landmark_14_z',
 'landmark_15_x',
 'landmark_15_y',
 'landmark_15_z',
 'landmark_16_x',
 'landmark_16_y',
 'landmark_16_z',
 'landmark_17_x',
 'landmark_17_y',
 'landmark_17_z',
 'landmark_18_x',
 'landmark_18_y',
 'landmark_18_z',
 'la

In [121]:
raw_landamrks

,frame,timestamp,landmarks
0,0,2025-06-16 11:47:55.476017,"[{""x"": 0.3988448679447174, ""y"": 0.265067487955..."
1,1,2025-06-16 11:47:55.518339,"[{""x"": 0.3968447744846344, ""y"": 0.326076626777..."
2,2,2025-06-16 11:47:55.646965,"[{""x"": 0.36895692348480225, ""y"": 0.38923734426..."
3,3,2025-06-16 11:47:55.768647,"[{""x"": 0.34413906931877136, ""y"": 0.42148470878..."
4,4,2025-06-16 11:47:55.892866,"[{""x"": 0.3308698832988739, ""y"": 0.420229792594..."
...,...,...,...
86,86,2025-06-16 11:48:05.732262,"[{""x"": 0.2568344473838806, ""y"": 0.641707003116..."
87,87,2025-06-16 11:48:05.849764,"[{""x"": 0.259283185005188, ""y"": 0.5849005579948..."
88,88,2025-06-16 11:48:05.976951,"[{""x"": 0.25951820611953735, ""y"": 0.54812550544..."
89,89,2025-06-16 11:48:06.090683,"[{""x"": 0.2624850571155548, ""y"": 0.501761853694..."


In [ ]:
landmarks= raw_landamrks['landmarks'].apply(lambda x: literal_eval(x)).apply(lambda x: extend_row(x))

In [129]:
landmarks_df = pd.DataFrame(landmarks.to_list(), columns=cols)

ValueError: 297 columns passed, passed data had 99 columns

In [124]:
landmarks_df

,landmark_0_x,landmark_0_y,landmark_0_z,landmark_1_x,landmark_1_y,landmark_1_z,landmark_2_x,landmark_2_y,landmark_2_z,landmark_3_x,...,landmark_29_z,landmark_30_x,landmark_30_y,landmark_30_z,landmark_31_x,landmark_31_y,landmark_31_z,landmark_32_x,landmark_32_y,landmark_32_z
0,0.398845,0.265067,0.046302,0.384576,0.251663,0.025139,0.384053,0.247401,0.025041,0.383676,...,0.021766,0.760857,0.706979,0.187713,0.662800,0.768926,-0.016232,0.691733,0.780575,0.145696
1,0.396845,0.326077,0.044797,0.380561,0.313218,0.021978,0.379682,0.309302,0.021961,0.378706,...,0.020850,0.773742,0.682830,0.197501,0.598971,0.755415,-0.012713,0.724026,0.751524,0.151067
2,0.368957,0.389237,0.056373,0.349553,0.384684,0.028380,0.348133,0.380971,0.028429,0.346916,...,-0.180925,0.695399,0.691064,0.158610,0.940120,0.724109,-0.231368,0.627977,0.756034,0.130403
3,0.344139,0.421485,0.002468,0.326559,0.410992,-0.029598,0.325660,0.407033,-0.029558,0.324649,...,-0.120402,0.688662,0.680136,0.177002,0.977585,0.700314,-0.201970,0.647967,0.736402,0.130947
4,0.330870,0.420230,0.046951,0.314334,0.411361,0.018544,0.313552,0.407935,0.018578,0.312806,...,-0.084855,0.827542,0.584779,0.155773,0.847773,0.706296,-0.149091,0.789948,0.682826,0.115794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,0.256834,0.641707,0.026650,0.243900,0.627489,0.008475,0.244202,0.622760,0.008383,0.244295,...,-0.106866,1.068647,0.599532,0.108945,1.113494,0.724383,-0.177212,1.084760,0.716751,-0.000895
87,0.259283,0.584901,-0.015021,0.246470,0.564173,-0.022431,0.247193,0.557494,-0.022575,0.247887,...,-0.118666,1.070159,0.593815,0.072425,1.117552,0.724601,-0.187594,1.089829,0.711993,-0.022702
88,0.259518,0.548126,-0.031868,0.246788,0.523465,-0.038514,0.247822,0.516425,-0.038671,0.248865,...,-0.129051,1.069046,0.594158,0.054243,1.110706,0.730697,-0.197856,1.037463,0.719453,-0.034124
89,0.262485,0.501762,-0.028071,0.249441,0.476821,-0.037258,0.250673,0.468735,-0.037403,0.251893,...,-0.171395,1.048963,0.594447,0.031466,1.085003,0.741227,-0.253413,1.006996,0.719880,-0.051337


In [125]:
landmarks_processed = pd.concat([raw_landamrks, landmarks_df], axis=1)

In [126]:
landmarks_processed.drop(columns=['landmarks'], inplace=True)

In [127]:
landmarks_processed

,frame,timestamp,landmark_0_x,landmark_0_y,landmark_0_z,landmark_1_x,landmark_1_y,landmark_1_z,landmark_2_x,landmark_2_y,...,landmark_29_z,landmark_30_x,landmark_30_y,landmark_30_z,landmark_31_x,landmark_31_y,landmark_31_z,landmark_32_x,landmark_32_y,landmark_32_z
0,0,2025-06-16 11:47:55.476017,0.398845,0.265067,0.046302,0.384576,0.251663,0.025139,0.384053,0.247401,...,0.021766,0.760857,0.706979,0.187713,0.662800,0.768926,-0.016232,0.691733,0.780575,0.145696
1,1,2025-06-16 11:47:55.518339,0.396845,0.326077,0.044797,0.380561,0.313218,0.021978,0.379682,0.309302,...,0.020850,0.773742,0.682830,0.197501,0.598971,0.755415,-0.012713,0.724026,0.751524,0.151067
2,2,2025-06-16 11:47:55.646965,0.368957,0.389237,0.056373,0.349553,0.384684,0.028380,0.348133,0.380971,...,-0.180925,0.695399,0.691064,0.158610,0.940120,0.724109,-0.231368,0.627977,0.756034,0.130403
3,3,2025-06-16 11:47:55.768647,0.344139,0.421485,0.002468,0.326559,0.410992,-0.029598,0.325660,0.407033,...,-0.120402,0.688662,0.680136,0.177002,0.977585,0.700314,-0.201970,0.647967,0.736402,0.130947
4,4,2025-06-16 11:47:55.892866,0.330870,0.420230,0.046951,0.314334,0.411361,0.018544,0.313552,0.407935,...,-0.084855,0.827542,0.584779,0.155773,0.847773,0.706296,-0.149091,0.789948,0.682826,0.115794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,86,2025-06-16 11:48:05.732262,0.256834,0.641707,0.026650,0.243900,0.627489,0.008475,0.244202,0.622760,...,-0.106866,1.068647,0.599532,0.108945,1.113494,0.724383,-0.177212,1.084760,0.716751,-0.000895
87,87,2025-06-16 11:48:05.849764,0.259283,0.584901,-0.015021,0.246470,0.564173,-0.022431,0.247193,0.557494,...,-0.118666,1.070159,0.593815,0.072425,1.117552,0.724601,-0.187594,1.089829,0.711993,-0.022702
88,88,2025-06-16 11:48:05.976951,0.259518,0.548126,-0.031868,0.246788,0.523465,-0.038514,0.247822,0.516425,...,-0.129051,1.069046,0.594158,0.054243,1.110706,0.730697,-0.197856,1.037463,0.719453,-0.034124
89,89,2025-06-16 11:48:06.090683,0.262485,0.501762,-0.028071,0.249441,0.476821,-0.037258,0.250673,0.468735,...,-0.171395,1.048963,0.594447,0.031466,1.085003,0.741227,-0.253413,1.006996,0.719880,-0.051337


In [2]:
import datetime

In [3]:
a=[{'frame': 22, 'timestamp': datetime.datetime(2025, 6, 16, 13, 31, 59, 9648), 'landmarks': [{'x': 0.9584019780158997, 'y': 0.6208194494247437}, {'x': 0.9775115847587585, 'y': 0.6119468808174133}, {'x': 0.9785365462303162, 'y': 0.612835168838501}, {'x': 0.9797853827476501, 'y': 0.6136283278465271}, {'x': 0.9766413569450378, 'y': 0.6026207208633423}, {'x': 0.9766911864280701, 'y': 0.5967347621917725}, {'x': 0.9767530560493469, 'y': 0.5905501842498779}, {'x': 0.9743056297302246, 'y': 0.5913425087928772}, {'x': 0.9647620916366577, 'y': 0.5611128211021423}, {'x': 0.9420913457870483, 'y': 0.616111695766449}, {'x': 0.9411618709564209, 'y': 0.6056241989135742}, {'x': 0.8872949481010437, 'y': 0.5116850733757019}, {'x': 0.8734560608863831, 'y': 0.5335323810577393}, {'x': 0.7700916528701782, 'y': 0.47708243131637573}, {'x': 0.7402681112289429, 'y': 0.5061434507369995}, {'x': 0.814599871635437, 'y': 0.5992345809936523}, {'x': 0.7989620566368103, 'y': 0.6700361967086792}, {'x': 0.8274697065353394, 'y': 0.6100226044654846}, {'x': 0.8294830322265625, 'y': 0.6907989978790283}, {'x': 0.8392699360847473, 'y': 0.620063304901123}, {'x': 0.8398013710975647, 'y': 0.6801378130912781}, {'x': 0.8281021118164062, 'y': 0.6186907291412354}, {'x': 0.8300290703773499, 'y': 0.6732385754585266}, {'x': 0.6027652025222778, 'y': 0.541447103023529}, {'x': 0.5926315784454346, 'y': 0.5668816566467285}, {'x': 0.41564124822616577, 'y': 0.5629223585128784}, {'x': 0.40073859691619873, 'y': 0.5812072157859802}, {'x': 0.24810411036014557, 'y': 0.5393187999725342}, {'x': 0.22289440035820007, 'y': 0.5519148707389832}, {'x': 0.2101355493068695, 'y': 0.5108451247215271}, {'x': 0.18463224172592163, 'y': 0.5175136923789978}, {'x': 0.2279786616563797, 'y': 0.6259192824363708}, {'x': 0.19817960262298584, 'y': 0.6454527378082275}]}, {'frame': 23, 'timestamp': datetime.datetime(2025, 6, 16, 13, 31, 59, 128219), 'landmarks': [{'x': 0.9554529190063477, 'y': 0.651782751083374}, {'x': 0.9740802049636841, 'y': 0.6420704126358032}, {'x': 0.9749650955200195, 'y': 0.6417485475540161}, {'x': 0.9760112166404724, 'y': 0.640600323677063}, {'x': 0.9726869463920593, 'y': 0.6343543529510498}, {'x': 0.9722868204116821, 'y': 0.6281335949897766}, {'x': 0.9718689918518066, 'y': 0.6216527223587036}, {'x': 0.9695009589195251, 'y': 0.6120645999908447}, {'x': 0.959775984287262, 'y': 0.5930487513542175}, {'x': 0.9416428208351135, 'y': 0.6473143100738525}, {'x': 0.9404453039169312, 'y': 0.6375781297683716}, {'x': 0.8851908445358276, 'y': 0.5456287264823914}, {'x': 0.8737457990646362, 'y': 0.5648269057273865}, {'x': 0.7746116518974304, 'y': 0.47984787821769714}, {'x': 0.7498970031738281, 'y': 0.5056746006011963}, {'x': 0.8133784532546997, 'y': 0.5993655323982239}, {'x': 0.7995339035987854, 'y': 0.670074462890625}, {'x': 0.8357106447219849, 'y': 0.6170836091041565}, {'x': 0.8295042514801025, 'y': 0.6912228465080261}, {'x': 0.839859664440155, 'y': 0.6260581612586975}, {'x': 0.8412079811096191, 'y': 0.6811873912811279}, {'x': 0.834295392036438, 'y': 0.6251062154769897}, {'x': 0.8328770399093628, 'y': 0.6741082072257996}, {'x': 0.6084714531898499, 'y': 0.5591877102851868}, {'x': 0.5986979007720947, 'y': 0.5715563297271729}, {'x': 0.41664260625839233, 'y': 0.5751628279685974}, {'x': 0.40489456057548523, 'y': 0.5849118828773499}, {'x': 0.24768774211406708, 'y': 0.546673595905304}, {'x': 0.2228061854839325, 'y': 0.5549520254135132}, {'x': 0.21064962446689606, 'y': 0.5170786380767822}, {'x': 0.18477876484394073, 'y': 0.5206480026245117}, {'x': 0.22563286125659943, 'y': 0.6318028569221497}, {'x': 0.19690118730068207, 'y': 0.6457507610321045}]}, {'frame': 24, 'timestamp': datetime.datetime(2025, 6, 16, 13, 31, 59, 252275), 'landmarks': [{'x': 0.9542722105979919, 'y': 0.6646231412887573}, {'x': 0.9722350239753723, 'y': 0.6590168476104736}, {'x': 0.9731848239898682, 'y': 0.6612924933433533}, {'x': 0.9742692112922668, 'y': 0.6631080508232117}, {'x': 0.9705846905708313, 'y': 0.642863929271698}, {'x': 0.9700065851211548, 'y': 0.6334426999092102}, {'x': 0.9693388342857361, 'y': 0.6228371262550354}, {'x': 0.9682397842407227, 'y': 0.6378350257873535}, {'x': 0.9575976133346558, 'y': 0.5916150808334351}, {'x': 0.9407607913017273, 'y': 0.6639336943626404}, {'x': 0.9386409521102905, 'y': 0.6463048458099365}, {'x': 0.8913045525550842, 'y': 0.585500180721283}, {'x': 0.8716049790382385, 'y': 0.5592591166496277}, {'x': 0.8042004704475403, 'y': 0.5239350199699402}, {'x': 0.7493780255317688, 'y': 0.5045573115348816}, {'x': 0.8218369483947754, 'y': 0.6097515225410461}, {'x': 0.8004193902015686, 'y': 0.6712023019790649}, {'x': 0.8328624963760376, 'y': 0.6271826028823853}, {'x': 0.8295366764068604, 'y': 0.6924891471862793}, {'x': 0.844088077545166, 'y': 0.6435032486915588}, {'x': 0.8418819904327393, 'y': 0.685888946056366}, {'x': 0.8403695225715637, 'y': 0.6426300406455994}, {'x': 0.8340606093406677, 'y': 0.6790701150894165}, {'x': 0.6066914796829224, 'y': 0.5801605582237244}, {'x': 0.597378134727478, 'y': 0.5688170194625854}, {'x': 0.4169471263885498, 'y': 0.5869854688644409}, {'x': 0.40817201137542725, 'y': 0.5868389010429382}, {'x': 0.24816825985908508, 'y': 0.5537959933280945}, {'x': 0.22306734323501587, 'y': 0.55703204870224}, {'x': 0.21229423582553864, 'y': 0.519650936126709}, {'x': 0.18554091453552246, 'y': 0.5208255052566528}, {'x': 0.22140121459960938, 'y': 0.6352784037590027}, {'x': 0.19230741262435913, 'y': 0.6446784734725952}]}, {'frame': 25, 'timestamp': datetime.datetime(2025, 6, 16, 13, 31, 59, 371056), 'landmarks': [{'x': 0.9533876776695251, 'y': 0.6333926320075989}, {'x': 0.9711448550224304, 'y': 0.6201736927032471}, {'x': 0.9719963669776917, 'y': 0.619979739189148}, {'x': 0.9730074405670166, 'y': 0.619769811630249}, {'x': 0.9699968099594116, 'y': 0.6125936508178711}, {'x': 0.9697747230529785, 'y': 0.6069732308387756}, {'x': 0.9696253538131714, 'y': 0.6007028818130493}, {'x': 0.9667059183120728, 'y': 0.5938165783882141}, {'x': 0.9587143063545227, 'y': 0.5732682347297668}, {'x': 0.9394717812538147, 'y': 0.6299793720245361}, {'x': 0.9384180903434753, 'y': 0.6219624876976013}, {'x': 0.8841056227684021, 'y': 0.5376378297805786}, {'x': 0.872516393661499, 'y': 0.5408288240432739}, {'x': 0.7735804319381714, 'y': 0.5072577595710754}, {'x': 0.7471476793289185, 'y': 0.5057081580162048}, {'x': 0.8187794089317322, 'y': 0.6101446151733398}, {'x': 0.8011658787727356, 'y': 0.6711528897285461}, {'x': 0.8152231574058533, 'y': 0.62928706407547}, {'x': 0.8310166001319885, 'y': 0.6924732327461243}, {'x': 0.8328986167907715, 'y': 0.644745409488678}, {'x': 0.8443243503570557, 'y': 0.6858107447624207}, {'x': 0.8351587057113647, 'y': 0.6373738646507263}, {'x': 0.8351755738258362, 'y': 0.6788893342018127}, {'x': 0.6064625382423401, 'y': 0.5532841682434082}, {'x': 0.5981570482254028, 'y': 0.5588294267654419}, {'x': 0.41745245456695557, 'y': 0.5791859030723572}, {'x': 0.4074684679508209, 'y': 0.5852182507514954}, {'x': 0.247482031583786, 'y': 0.5545909404754639}, {'x': 0.2229316532611847, 'y': 0.5575159788131714}, {'x': 0.21251171827316284, 'y': 0.5212677121162415}, {'x': 0.1856902837753296, 'y': 0.5211569666862488}, {'x': 0.221595898270607, 'y': 0.6350862979888916}, {'x': 0.1915970742702484, 'y': 0.6445144414901733}]}, {'frame': 26, 'timestamp': datetime.datetime(2025, 6, 16, 13, 31, 59, 485308), 'landmarks': [{'x': 0.9576773047447205, 'y': 0.596260666847229}, {'x': 0.9757007360458374, 'y': 0.5834702849388123}, {'x': 0.9765128493309021, 'y': 0.5834612250328064}, {'x': 0.9774296283721924, 'y': 0.5834515690803528}, {'x': 0.9750905632972717, 'y': 0.5755172371864319}, {'x': 0.9752960205078125, 'y': 0.5697502493858337}, {'x': 0.9755134582519531, 'y': 0.5634731650352478}, {'x': 0.9733328819274902, 'y': 0.5600491762161255}, {'x': 0.9680274724960327, 'y': 0.5352603197097778}, {'x': 0.9434388279914856, 'y': 0.5937665104866028}, {'x': 0.9426366090774536, 'y': 0.5848953127861023}, {'x': 0.889245867729187, 'y': 0.5292937755584717}, {'x': 0.8787618279457092, 'y': 0.5069997310638428}, {'x': 0.7701847553253174, 'y': 0.5338836312294006}, {'x': 0.7503599524497986, 'y': 0.5104121565818787}, {'x': 0.8150491714477539, 'y': 0.6265044808387756}, {'x': 0.8014635443687439, 'y': 0.6714127659797668}, {'x': 0.8091917037963867, 'y': 0.6341705918312073}, {'x': 0.8319891095161438, 'y': 0.6945865154266357}, {'x': 0.8315693140029907, 'y': 0.6436638832092285}, {'x': 0.8457692265510559, 'y': 0.6860141158103943}, {'x': 0.832516074180603, 'y': 0.640696108341217}, {'x': 0.8351452350616455, 'y': 0.6788595914840698}, {'x': 0.6094940900802612, 'y': 0.5480377674102783}, {'x': 0.6029412150382996, 'y': 0.5408855080604553}, {'x': 0.42077142000198364, 'y': 0.5763205289840698}, {'x': 0.4106903076171875, 'y': 0.5771127343177795}, {'x': 0.24778537452220917, 'y': 0.554162859916687}, {'x': 0.2236143797636032, 'y': 0.5574655532836914}, {'x': 0.21257366240024567, 'y': 0.5216756463050842}, {'x': 0.18787434697151184, 'y': 0.5214135646820068}, {'x': 0.22388261556625366, 'y': 0.6350408792495728}, {'x': 0.19140660762786865, 'y': 0.6440049409866333}]}, {'frame': 27, 'timestamp': datetime.datetime(2025, 6, 16, 13, 31, 59, 614736), 'landmarks': [{'x': 0.9685110449790955, 'y': 0.5425045490264893}, {'x': 0.9871719479560852, 'y': 0.5319098234176636}, {'x': 0.9873532056808472, 'y': 0.5332807302474976}, {'x': 0.9876492619514465, 'y': 0.5347301959991455}, {'x': 0.9871528744697571, 'y': 0.5224873423576355}, {'x': 0.9870907068252563, 'y': 0.5169757604598999}, {'x': 0.9868729710578918, 'y': 0.5104402303695679}, {'x': 0.981708288192749, 'y': 0.5196015238761902}, {'x': 0.9773576259613037, 'y': 0.48565471172332764}, {'x': 0.9519723653793335, 'y': 0.5433945655822754}, {'x': 0.951805055141449, 'y': 0.5317659974098206}, {'x': 0.8854312300682068, 'y': 0.5071765780448914}, {'x': 0.8882075548171997, 'y': 0.46916747093200684}, {'x': 0.7587941288948059, 'y': 0.5248140096664429}, {'x': 0.7655794620513916, 'y': 0.5125279426574707}, {'x': 0.7914744019508362, 'y': 0.6224974393844604}, {'x': 0.8014540076255798, 'y': 0.6711239814758301}, {'x': 0.8058156967163086, 'y': 0.6309983730316162}, {'x': 0.8318287134170532, 'y': 0.6945213079452515}, {'x': 0.8251675963401794, 'y': 0.6422228217124939}, {'x': 0.8447751402854919, 'y': 0.6845505237579346}, {'x': 0.8227723240852356, 'y': 0.6360614895820618}, {'x': 0.8345459699630737, 'y': 0.6771039366722107}, {'x': 0.6086635589599609, 'y': 0.5352916717529297}, {'x': 0.6044424772262573, 'y': 0.5010536313056946}, {'x': 0.4234766662120819, 'y': 0.5704025626182556}, {'x': 0.41225069761276245, 'y': 0.5645891427993774}, {'x': 0.2479131519794464, 'y': 0.5535954833030701}, {'x': 0.22494079172611237, 'y': 0.5569695234298706}, {'x': 0.21527159214019775, 'y': 0.5217347741127014}, {'x': 0.18876603245735168, 'y': 0.5215296149253845}, {'x': 0.2230163961648941, 'y': 0.6356757879257202}, {'x': 0.19049909710884094, 'y': 0.6441729664802551}]}, {'frame': 28, 'timestamp': datetime.datetime(2025, 6, 16, 13, 31, 59, 725756), 'landmarks': [{'x': 0.9666576981544495, 'y': 0.4858873188495636}, {'x': 0.9848502278327942, 'y': 0.47044283151626587}, {'x': 0.9850637912750244, 'y': 0.46908605098724365}, {'x': 0.9854011535644531, 'y': 0.4677470028400421}, {'x': 0.9848251342773438, 'y': 0.46675604581832886}, {'x': 0.9848164916038513, 'y': 0.462444543838501}, {'x': 0.9847828149795532, 'y': 0.45773014426231384}, {'x': 0.9787201881408691, 'y': 0.44249266386032104}, {'x': 0.9745188355445862, 'y': 0.43024346232414246}, {'x': 0.9501832127571106, 'y': 0.48166796565055847}, {'x': 0.9498497843742371, 'y': 0.47543391585350037}, {'x': 0.8771331906318665, 'y': 0.4178469479084015}, {'x': 0.8864707350730896, 'y': 0.4154394865036011}, {'x': 0.7581363916397095, 'y': 0.5059018135070801}, {'x': 0.7727965712547302, 'y': 0.5129984021186829}, {'x': 0.7884096503257751, 'y': 0.6206338405609131}, {'x': 0.8014450073242188, 'y': 0.6743530035018921}, {'x': 0.8066439032554626, 'y': 0.6179018616676331}, {'x': 0.8338356614112854, 'y': 0.6947275996208191}, {'x': 0.8250291347503662, 'y': 0.6280500888824463}, {'x': 0.8448759913444519, 'y': 0.6846970319747925}, {'x': 0.8223705887794495, 'y': 0.6284293532371521}, {'x': 0.8344109654426575, 'y': 0.6772957444190979}, {'x': 0.6077150702476501, 'y': 0.4768439531326294}, {'x': 0.6047741770744324, 'y': 0.4756443202495575}, {'x': 0.42357826232910156, 'y': 0.5494686961174011}, {'x': 0.41221922636032104, 'y': 0.5492048859596252}, {'x': 0.24713872373104095, 'y': 0.552862823009491}, {'x': 0.2248719036579132, 'y': 0.5568411350250244}, {'x': 0.21487808227539062, 'y': 0.5222972631454468}, {'x': 0.1879356950521469, 'y': 0.522894024848938}, {'x': 0.22421494126319885, 'y': 0.6355765461921692}, {'x': 0.19079625606536865, 'y': 0.6452631950378418}]}, {'frame': 29, 'timestamp': datetime.datetime(2025, 6, 16, 13, 31, 59, 854843), 'landmarks': [{'x': 0.9661725163459778, 'y': 0.40585312247276306}, {'x': 0.9840731024742126, 'y': 0.39045292139053345}, {'x': 0.9840959310531616, 'y': 0.38873526453971863}, {'x': 0.9843161702156067, 'y': 0.3875124752521515}, {'x': 0.9839404821395874, 'y': 0.38465362787246704}, {'x': 0.9836717247962952, 'y': 0.3791815936565399}, {'x': 0.9834309816360474, 'y': 0.37396904826164246}, {'x': 0.9769881963729858, 'y': 0.3646855652332306}, {'x': 0.9720233678817749, 'y': 0.34693583846092224}, {'x': 0.9485815167427063, 'y': 0.40470048785209656}, {'x': 0.947910726070404, 'y': 0.398330420255661}, {'x': 0.8627623915672302, 'y': 0.3477514088153839}, {'x': 0.8855626583099365, 'y': 0.35152000188827515}, {'x': 0.7647998332977295, 'y': 0.4835978150367737}, {'x': 0.7846686244010925, 'y': 0.5072259306907654}, {'x': 0.7838014960289001, 'y': 0.6186161041259766}, {'x': 0.8009266257286072, 'y': 0.6742319464683533}, {'x': 0.8066173195838928, 'y': 0.6209760904312134}, {'x': 0.8336464166641235, 'y': 0.6946923136711121}, {'x': 0.822404146194458, 'y': 0.6281510591506958}, {'x': 0.8439435362815857, 'y': 0.6846884489059448}, {'x': 0.8166607618331909, 'y': 0.6283469200134277}, {'x': 0.8335404396057129, 'y': 0.6775514483451843}, {'x': 0.6073808670043945, 'y': 0.43094754219055176}, {'x': 0.6041982173919678, 'y': 0.4330021142959595}, {'x': 0.4237464964389801, 'y': 0.5209681391716003}, {'x': 0.41343531012535095, 'y': 0.522266149520874}, {'x': 0.24311797320842743, 'y': 0.552460253238678}, {'x': 0.22473035752773285, 'y': 0.5560635328292847}, {'x': 0.21037539839744568, 'y': 0.524208664894104}, {'x': 0.18571844696998596, 'y': 0.5234509110450745}, {'x': 0.2268614023923874, 'y': 0.6385546922683716}, {'x': 0.19093424081802368, 'y': 0.6471512317657471}]}, {'frame': 30, 'timestamp': datetime.datetime(2025, 6, 16, 13, 31, 59, 962029), 'landmarks': [{'x': 0.9669741988182068, 'y': 0.33105674386024475}, {'x': 0.9841305017471313, 'y': 0.3201691508293152}, {'x': 0.9845426082611084, 'y': 0.32270169258117676}, {'x': 0.9850899577140808, 'y': 0.32487624883651733}, {'x': 0.9827495813369751, 'y': 0.30482327938079834}, {'x': 0.9822413921356201, 'y': 0.29629233479499817}, {'x': 0.982200026512146, 'y': 0.28687548637390137}, {'x': 0.9771088361740112, 'y': 0.30899694561958313}, {'x': 0.9715672135353088, 'y': 0.26401016116142273}, {'x': 0.9479490518569946, 'y': 0.3436298668384552}, {'x': 0.9469580054283142, 'y': 0.3275162875652313}, {'x': 0.854203462600708, 'y': 0.3061901926994324}, {'x': 0.879734992980957, 'y': 0.3077258765697479}, {'x': 0.7712782025337219, 'y': 0.46790286898612976}, {'x': 0.7989839315414429, 'y': 0.496389776468277}, {'x': 0.7775561213493347, 'y': 0.6175541877746582}, {'x': 0.8009167313575745, 'y': 0.6794148087501526}, {'x': 0.8026255369186401, 'y': 0.6206413507461548}, {'x': 0.8336203098297119, 'y': 0.6993135213851929}, {'x': 0.8130043148994446, 'y': 0.6284807920455933}, {'x': 0.8437624573707581, 'y': 0.6910508275032043}, {'x': 0.801307737827301, 'y': 0.627953827381134}, {'x': 0.83287513256073, 'y': 0.6833574771881104}, {'x': 0.6016179323196411, 'y': 0.39722058176994324}, {'x': 0.6010924577713013, 'y': 0.39822763204574585}, {'x': 0.41973191499710083, 'y': 0.5018698573112488}, {'x': 0.41299688816070557, 'y': 0.5005786418914795}, {'x': 0.23871061205863953, 'y': 0.5504214763641357}, {'x': 0.22465285658836365, 'y': 0.5539501905441284}, {'x': 0.2049969583749771, 'y': 0.5241541266441345}, {'x': 0.1850883960723877, 'y': 0.5235294699668884}, {'x': 0.22664211690425873, 'y': 0.6397106647491455}, {'x': 0.19547036290168762, 'y': 0.6475077271461487}]}]

In [4]:
df = pd.DataFrame(a)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   frame      9 non-null      int64         
 1   timestamp  9 non-null      datetime64[ns]
 2   landmarks  9 non-null      object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 348.0+ bytes


In [25]:
def extend_row(row: dict) -> list:
    """
    Extend a row of landmarks to a flat list.
    Args:
        row (list): A list of landmarks, where each landmark is a list of [x, y, z].
    Returns:
        list: A flat list of landmarks in the format [landmark_0_x, landmark_0_y, ..., landmark_n_x, landmark_n_y].
    """
    extended_row = []
    for landmark in row:
        extended_row.extend([landmark['x'], landmark['y']])
    return extended_row

In [26]:
df['landmarks'].apply(lambda x: extend_row(x))

0    [0.9584019780158997, 0.6208194494247437, 0.977...
1    [0.9554529190063477, 0.651782751083374, 0.9740...
2    [0.9542722105979919, 0.6646231412887573, 0.972...
3    [0.9533876776695251, 0.6333926320075989, 0.971...
4    [0.9576773047447205, 0.596260666847229, 0.9757...
5    [0.9685110449790955, 0.5425045490264893, 0.987...
6    [0.9666576981544495, 0.4858873188495636, 0.984...
7    [0.9661725163459778, 0.40585312247276306, 0.98...
8    [0.9669741988182068, 0.33105674386024475, 0.98...
Name: landmarks, dtype: object